In [1]:
import pymongo
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# create the connection to MongoDB
# create an instance of the database and collections

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.australia_fire_db
bushfiresbyState = db.bushfiresbyState
historicalFires = db.historicalFires

In [3]:
# urls to scrape

byStateurl = 'https://en.wikipedia.org/wiki/List_of_fires_and_impacts_of_the_2019-20_Australian_bushfire_season'
historicalurl = 'https://en.wikipedia.org/wiki/List_of_major_bushfires_in_Australia'

In [4]:
# request the html using beautiful soup

historical_response = requests.get(historicalurl)

In [5]:
# parse the html text

h_soup = BeautifulSoup(historical_response.text, 'html.parser')

In [6]:
# the table headers are within the table body in the first two rows

h_table = h_soup.find('table', class_="wikitable")
h_table_headers = h_soup.find_all("tr")[0:2]

In [7]:
# scrape the table headers and append them to the headers list

h_headers = []
for table_header in h_table_headers:
    try:
        value = table_header.text
#         value = value.split("\n")
        h_headers.append(value)
        
    except AttributeError as e:
        print(e)
        
print(h_headers)
print(len(h_headers))

['\nDate\n\nName or description\n\nState(s) /territories\n\nArea burned (approx.)\n\nFatalities\n\nProperties damaged\n\nNotes\n', '\nha\n\nacres\n\nHomes(destroyed)\n\nOther buildings\n\nOther damage\n']
2


In [8]:
# manually create the headers list, because the scraped table headers was too complicated
# if there is time, create logic to put the two header rows into one similar to the list below

h_headers = ['Date', 'Name', 'State(s)/territories', 'AreaBurned(ha)', 'AreaBurned(acres)', 'Fatalities', 'PropertiesDamaged(HomesDestroyed)']

In [9]:
# find objects to scrape
# the table data is after the headers which is in the first two rows

h_table = h_soup.find('table', class_="wikitable")
h_table_body = h_table.find("tbody")
h_table_row = h_table_body.find_all('tr')[2:]

In [10]:
# scrape data and create a list of lists for each row of data

h_data = []

for row in h_table_row:
    
    datarow = []
    
    table_data = row.find_all('td')
    
    for tdata in table_data:
        try:
            value = tdata.text
    #             value.split("\n\n")
            value = value.replace("\n", "")
            datarow.append(value)

        except AttributeError as e:
            print(e)
            
    h_data.append(datarow)
        
# print(h_data)

In [11]:
# convert scrapped data into a dataframe

h_df = pd.DataFrame(h_data)
h_df

,0,1,2,3,4,5,6,7,8,9
0,6 February 1851,Black Thursday bushfires,Victoria,"5,000,000","12,000,000",approx. 12,Nil,Nil,1 million sheepthousands of cattle,[7][8]
1,1 February 1898,Red Tuesday bushfires,Victoria,"260,000","640,000",12,Nil,"2,000",,[8][9][10]
2,February – March 1926,1926 bushfires,Victoria,"390,000","960,000",60,"1,000",Nil,,[11]
3,December 1938 – January 1939,Black Friday bushfires,Victoria,"2,000,000","4,900,000",71,"3,700",Nil,,[5]
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,"1,000,000","2,500,000",15–20,approx. 500,Nil,,[8]
...,...,...,...,...,...,...,...,...,...,...
63,25 November – 2 December 2015,2015 Pinery bushfire,South Australia,"85,000","210,000",2,91,Nil,,[66][67]
64,January 2016,2016 Murray Road bushfire (Waroona and Harvey),Western Australia,"69,165","170,910",2,181,Yarloop Workshops,Thousands of hectares of Lane Poole ReservePro...,[68]
65,11 – 14 February 2017,2017 New South Wales bushfires,New South Wales,"52,000","130,000",Nil,35,Nil,,[69]
66,early February 2019,Tingha bushfire,New South Wales,"23,419","57,870",Nil,19,57,LivestockVineyards,[70][71]


In [12]:
# keep desired rows

h_drop_rows_df = h_df[[0,1,2,3,4,5,6]]
h_drop_rows_df

,0,1,2,3,4,5,6
0,6 February 1851,Black Thursday bushfires,Victoria,"5,000,000","12,000,000",approx. 12,Nil
1,1 February 1898,Red Tuesday bushfires,Victoria,"260,000","640,000",12,Nil
2,February – March 1926,1926 bushfires,Victoria,"390,000","960,000",60,"1,000"
3,December 1938 – January 1939,Black Friday bushfires,Victoria,"2,000,000","4,900,000",71,"3,700"
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,"1,000,000","2,500,000",15–20,approx. 500
...,...,...,...,...,...,...,...
63,25 November – 2 December 2015,2015 Pinery bushfire,South Australia,"85,000","210,000",2,91
64,January 2016,2016 Murray Road bushfire (Waroona and Harvey),Western Australia,"69,165","170,910",2,181
65,11 – 14 February 2017,2017 New South Wales bushfires,New South Wales,"52,000","130,000",Nil,35
66,early February 2019,Tingha bushfire,New South Wales,"23,419","57,870",Nil,19


In [13]:
# Add column headers

h_drop_rows_df.columns = h_headers
h_drop_rows_df

,Date,Name,State(s)/territories,AreaBurned(ha),AreaBurned(acres),Fatalities,PropertiesDamaged(HomesDestroyed)
0,6 February 1851,Black Thursday bushfires,Victoria,"5,000,000","12,000,000",approx. 12,Nil
1,1 February 1898,Red Tuesday bushfires,Victoria,"260,000","640,000",12,Nil
2,February – March 1926,1926 bushfires,Victoria,"390,000","960,000",60,"1,000"
3,December 1938 – January 1939,Black Friday bushfires,Victoria,"2,000,000","4,900,000",71,"3,700"
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,"1,000,000","2,500,000",15–20,approx. 500
...,...,...,...,...,...,...,...
63,25 November – 2 December 2015,2015 Pinery bushfire,South Australia,"85,000","210,000",2,91
64,January 2016,2016 Murray Road bushfire (Waroona and Harvey),Western Australia,"69,165","170,910",2,181
65,11 – 14 February 2017,2017 New South Wales bushfires,New South Wales,"52,000","130,000",Nil,35
66,early February 2019,Tingha bushfire,New South Wales,"23,419","57,870",Nil,19


In [14]:
# replace Nil with 0 values

h_nil_df = h_drop_rows_df.replace('Nil', '0')
h_nil_df.head(10)

,Date,Name,State(s)/territories,AreaBurned(ha),AreaBurned(acres),Fatalities,PropertiesDamaged(HomesDestroyed)
0,6 February 1851,Black Thursday bushfires,Victoria,"5,000,000","12,000,000",approx. 12,0
1,1 February 1898,Red Tuesday bushfires,Victoria,"260,000","640,000",12,0
2,February – March 1926,1926 bushfires,Victoria,"390,000","960,000",60,"1,000"
3,December 1938 – January 1939,Black Friday bushfires,Victoria,"2,000,000","4,900,000",71,"3,700"
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,"1,000,000","2,500,000",15–20,approx. 500
5,18 November 1944,1944 Blue Mountains bushfire,New South Wales,,,0,approx. 40
6,November 1951 – January 1952,1951–52 bushfires,Victoria,"4,000,000","9,900,000",11,0
7,2 January 1955,Black Sunday bushfires,South Australia,"39,000–160,000","96,000–395,000",2,40[b]
8,30 November 1957,"1957 Grose Valley bushfire, Blue Mountains",New South Wales,,,4,0
9,2 December 1957,"1957 Leura bushfire, Blue Mountains",New South Wales,,,0,170[c]


In [15]:
# remove commas from numbers
# replace unwanted values as NaNs
# replace range data into an average value
# remove citations found within []

h_tonumeric_df = h_nil_df.copy()
h_tonumeric_df['AreaBurned(ha)'] = h_nil_df['AreaBurned(ha)'].str.replace(',','')
h_tonumeric_df['AreaBurned(acres)'] = h_nil_df['AreaBurned(acres)'].str.replace(',','')
h_tonumeric_df['PropertiesDamaged(HomesDestroyed)'] = h_nil_df['PropertiesDamaged(HomesDestroyed)'].str.replace(',','')

columnstoedit = ['AreaBurned(ha)','AreaBurned(acres)','Fatalities','PropertiesDamaged(HomesDestroyed)']

for i in range(len(h_tonumeric_df['Date'])):
    for column in columnstoedit:
        
        if ((h_tonumeric_df[column][i] == '') and (h_tonumeric_df[column][i] != '0')) or (h_tonumeric_df[column][i] == 'unknown') :
            h_tonumeric_df[column][i] = 'NaN'

        if 'approx. ' in str(h_tonumeric_df[column][i]):
            h_tonumeric_df[column][i] = h_tonumeric_df[column][i].replace('approx. ', '')
            
        if 'than ' in str(h_tonumeric_df[column][i]):
            h_tonumeric_df[column][i] = h_tonumeric_df[column][i].split(' ')[-1]

        if len(str(h_tonumeric_df[column][i]).split('–')) == 2:
            splitvalues = str(h_tonumeric_df[column][i]).split('–')
            h_tonumeric_df[column][i] = np.mean([int(splitvalues[0]), int(splitvalues[1])])

        if len(str(h_tonumeric_df[column][i]).split('[')) >= 2:
            h_tonumeric_df[column][i] = str(h_tonumeric_df[column][i]).split('[')[0]
        

h_tonumeric_df

,Date,Name,State(s)/territories,AreaBurned(ha),AreaBurned(acres),Fatalities,PropertiesDamaged(HomesDestroyed)
0,6 February 1851,Black Thursday bushfires,Victoria,5000000,12000000,12,0
1,1 February 1898,Red Tuesday bushfires,Victoria,260000,640000,12,0
2,February – March 1926,1926 bushfires,Victoria,390000,960000,60,1000
3,December 1938 – January 1939,Black Friday bushfires,Victoria,2000000,4900000,71,3700
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,1000000,2500000,17.5,500
...,...,...,...,...,...,...,...
63,25 November – 2 December 2015,2015 Pinery bushfire,South Australia,85000,210000,2,91
64,January 2016,2016 Murray Road bushfire (Waroona and Harvey),Western Australia,69165,170910,2,181
65,11 – 14 February 2017,2017 New South Wales bushfires,New South Wales,52000,130000,0,35
66,early February 2019,Tingha bushfire,New South Wales,23419,57870,0,19


In [16]:
# pull out the year the fire took place

h_year_df = h_tonumeric_df.copy()
h_year_df['Year'] = ''
for i in range(len(h_year_df['Date'])):

    if len(str(h_year_df['Date'][i]).split(' – ')) >= 2:
        results = str(h_year_df['Date'][i]).split(' – ')
        for result in results:
            if len(str(result).split()) == 4:
                year = result
    
    elif len(str(h_year_df['Date'][i]).split('[')) >= 2:
        year = str(h_year_df['Date'][i]).split('[')[0]
        
#     elif len(str(h_year_df['Date'][i]).split(' – ')) >= 2:
#         results = str(h_year_df['Date'][i]).split(' – ')
#         for result in results:
#             if len(str(result).split()) == 4:
#                 year = result
    
    else:
        year = str(h_year_df['Date'][i]).split(' ')[-1]
        
    h_year_df['Year'][i] = year

    
h_year_df

,Date,Name,State(s)/territories,AreaBurned(ha),AreaBurned(acres),Fatalities,PropertiesDamaged(HomesDestroyed),Year
0,6 February 1851,Black Thursday bushfires,Victoria,5000000,12000000,12,0,1851
1,1 February 1898,Red Tuesday bushfires,Victoria,260000,640000,12,0,1898
2,February – March 1926,1926 bushfires,Victoria,390000,960000,60,1000,1898
3,December 1938 – January 1939,Black Friday bushfires,Victoria,2000000,4900000,71,3700,1898
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,1000000,2500000,17.5,500,1898
...,...,...,...,...,...,...,...,...
63,25 November – 2 December 2015,2015 Pinery bushfire,South Australia,85000,210000,2,91,2015
64,January 2016,2016 Murray Road bushfire (Waroona and Harvey),Western Australia,69165,170910,2,181,2016
65,11 – 14 February 2017,2017 New South Wales bushfires,New South Wales,52000,130000,0,35,2016
66,early February 2019,Tingha bushfire,New South Wales,23419,57870,0,19,2019


In [17]:
# remove nan values and empty values and convert them to integers

h_casting_df = h_year_df.copy()

h_casting_df = h_casting_df[h_casting_df['AreaBurned(ha)'] != 'NaN']
h_casting_df = h_casting_df[h_casting_df['PropertiesDamaged(HomesDestroyed)'] != 'NaN']
h_casting_df = h_casting_df[h_casting_df['PropertiesDamaged(HomesDestroyed)'] != '']
h_casting_df = h_casting_df.astype(
    {
        'AreaBurned(ha)':'int',
        'AreaBurned(acres)': 'int',
        'Fatalities':'int',
        'PropertiesDamaged(HomesDestroyed)':'int',
        'Year': 'int'
    })
# h_casting_df['Year'] = pd.to_datetime(h_casting_df['Year'], format = '%Y')
h_casting_df.dtypes
h_casting_df

,Date,Name,State(s)/territories,AreaBurned(ha),AreaBurned(acres),Fatalities,PropertiesDamaged(HomesDestroyed),Year
0,6 February 1851,Black Thursday bushfires,Victoria,5000000,12000000,12,0,1851
1,1 February 1898,Red Tuesday bushfires,Victoria,260000,640000,12,0,1898
2,February – March 1926,1926 bushfires,Victoria,390000,960000,60,1000,1898
3,December 1938 – January 1939,Black Friday bushfires,Victoria,2000000,4900000,71,3700,1898
4,14 January – 14 February 1944,1944 Victorian bushfires,Victoria,1000000,2500000,17,500,1898
6,November 1951 – January 1952,1951–52 bushfires,Victoria,4000000,9900000,11,0,1944
7,2 January 1955,Black Sunday bushfires,South Australia,99500,245500,2,40,1955
10,January – March 1961,1961 Western Australian bushfires,Western Australia,1800000,4400000,0,160,1957
12,16 February – 13 March 1965,1965 Gippsland bushfires,Victoria,315000,780000,0,20,1957
13,5 – 14 March 1965,Southern Highlands bushfires,New South Wales,251000,620000,3,59,1957


In [18]:
# transform the dataframe into a dictionary so that we can feed it to MongoDB
h_dict = h_casting_df.to_dict('records')
h_dict

[{'Date': '6 February 1851',
  'Name': 'Black Thursday bushfires',
  'State(s)/territories': 'Victoria',
  'AreaBurned(ha)': 5000000,
  'AreaBurned(acres)': 12000000,
  'Fatalities': 12,
  'PropertiesDamaged(HomesDestroyed)': 0,
  'Year': 1851},
 {'Date': '1 February 1898',
  'Name': 'Red Tuesday bushfires',
  'State(s)/territories': 'Victoria',
  'AreaBurned(ha)': 260000,
  'AreaBurned(acres)': 640000,
  'Fatalities': 12,
  'PropertiesDamaged(HomesDestroyed)': 0,
  'Year': 1898},
 {'Date': 'February – March 1926',
  'Name': '1926 bushfires',
  'State(s)/territories': 'Victoria',
  'AreaBurned(ha)': 390000,
  'AreaBurned(acres)': 960000,
  'Fatalities': 60,
  'PropertiesDamaged(HomesDestroyed)': 1000,
  'Year': 1898},
 {'Date': 'December 1938 – January 1939',
  'Name': 'Black Friday bushfires',
  'State(s)/territories': 'Victoria',
  'AreaBurned(ha)': 2000000,
  'AreaBurned(acres)': 4900000,
  'Fatalities': 71,
  'PropertiesDamaged(HomesDestroyed)': 3700,
  'Year': 1898},
 {'Date': '14

In [27]:
# insert records into the MongoDB collection histroicalFires

if (historicalFires.find({}).count() == 0):
    historicalFires.insert(h_dict)
    
else:
    print("Data already exists")

/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused